# COGS 118A - Final Project

# IMDb Movies: Gross Income Prediction for Box Office Earnings

# Names


- Carly Freedman
- Jackson Teel
- Ye Yint Win
- Garrett Dungca

# Abstract 

We would like to create a program to predict the gross income in U.S. dollars of movies based on their year, certificate, runtime, genre, rating and total number of votes. The certificate gives the movie rating (i.e. PG-13), the rating represents the total score received by the movies from reviewers on IMDB, and the total number of votes is how many people reviewed the movie total. We will be using a dataset containing all of these variables for every movie on IMDB. We are going to one-hot encode each of the categories. In order to predict the gross income we will be using and comparing several regression models such as decision tree regression, LASSO, Ridge, and linear regression. We will use a loss value based on the actual gross income vs. predicted gross income.

# Background

There have been many studies conducted on predicting gross income of movies. However, there are few accurate ones. Most studies produce an accuracy in the 50-60% range. For example, a study conducted on this topic using both standard linear regression and classification via logistic regression only produced an accuracy of about 75% <a name="yoo"></a>[<sup>[1]</sup>](#yoonote). Given that this study was likely limited by the types of analysis they performed, we would like to determine if there is any way to increase this accuracy using a wider variety of algorithms and a different dataset. In our own analysis, we will likely use standard linear regression as well, but as a baseline reference for our other algorithms.

Using machine learning algorithms to predict gross income of movies has several benefits. Accurate income predictions can help movie studios, producers, and investors make better business decisions. It allows for more informed decisions to be made with regard to budgeting, marketing, and distribution<a name="dhir"></a>[<sup>[2]</sup>](#dhirnote). These predictions can also help filmmakers and studios create movies that are more engaging to their audience. ML algorithms can be used to analyze audience preferences based on demographics and other key variables that help inform creative decisions. From the article titled, ‘Popularity prediction of movies: from statistical modeling to machine learning techniques’, we were inspired to use a Decision Tree model for our predictions as it turned out to be one of the more accurate models in their case. <a name="abidi"></a>[<sup>[3]</sup>](#abidinote).

# Problem Statement

Our problem is how to predict the gross box office earnings of a movie based on features such as genre, rating, runtime, certificate, year, and votes. The metric we aim to optimize is the loss of the amount in US dollars of gross box office earnings compared with the predicted values, using IMDB's labeled dataset of movies. Our specific loss is undetermined at the moment, it is more likely that we will explore a variety of losses given the wide range of earnings a movie can make. This includes but is not limited to:
MAPE: Good for movies earning greater amounts even if the prediction is off, worse for predictions with low earning movies (proportion is greater with smaller numbers)
MSE: Useful but difficult to interpret especially with heavy mispredictions (could choose to look at earnings in # of millions of dollars instead)
MAE: Simple but doesn’t punish bad predictions as obviously as MSE
RMSE: Addresses the issue of earnings being in a large number amount but more difficult to interpret

Given that our model provides reliable information when predicting gross box office earnings, the hope is for our model to help movie studios, producers, and investors to make more informed business decisions about budgeting, marketing, and distribution of their movies. Alternatively, our analysis may reveal that the features we used are not informative enough, even given a variety of complex models and approaches, in which we would determine that the data obtained from IMDb should not be used alone in future research and may need to include more complex features or deeper additional analysis such as sentiment analysis of reviews. Additionally, if we determine that our models’ accuracy stagnate even as we add more complexity (accounting for methods to prevent overfitting), we may conclude that the features we did use poorly map onto box office earnings, suggesting that IMDb data is wholly uninformative when determining the financial success of a movie. As stated in the problem statement, we will likely explore multiple kinds of losses/error metrics, mostly because individually each metric can be both good and bad based on our data. In the first reference (“Predicting Movie Revenue from IMDb Data”), MAPE is particularly noted to be a poor metric because of the issue of lower earning movies having greater error on worse mispredictions [1]. Their data, however, only looks at 4052 movies whereas we have over 18000, meaning there is a possibility that their analysis may be not entirely informed.

# Data
Link to dataset: <a>https://www.kaggle.com/datasets/rajugc/imdb-movies-dataset-based-on-genre?resource=download</a>

Column Descriptions:

 • movie_id - IMDB Movie ID

 • movie_name - Name of the movie

 • year - Release year

 • certificate - Certificate of the movie

 • run_time - Total movie run time

 • genre - Genre of the movie

 • rating - Rating of the movie

 • description - Description of the movie

 • director - Director of the movie

 • director_id - IMDB id of the director

 • star - Star of the movie

 • star_id - IMDB id of the star

 • votes - Number of votes in IMDB website
 
 • gross(in $) - Gross Box Office of the movie


The dataset starts with 298975 total observations, and 14 variables. Once we remove the observations with Null values, we have 18709 total observations. Additionally, we are only interested in 6 of the variables as aforementioned because several of the variables are string values that would be too lengthy to one-hot encode. A valid observation must include the features: year (integer), certificate, runtime (integer), genre, rating (float value),  votes (integer), and gross income (float), and it must also have no null values. These variables are a combination of ordinal variables and categorical variables which are able to be one-hot encoded. Therefore they will be usable by our program.

Many of the movies in the dataset are listed under several genres. In total there are 40 unique genres listed in the dataset, but the data is organized into 13 folders: one for each main genre of movie. To save some computational expense in feature selection, we are going to only use the genre that corresponds to the directory that the movie was initially listed under. For the same reason, we will chunk the year data into groups of 15 years. 

The variables that we will one-hot encode will include genre and certificate. There are 27 unique certificates and 13 unique genres in our dataset that we are interested in (action, adventure, crime, family, fantasy, film-noir, history, horror, mystery, scifi, sports, thriller, and war). We can also one-hot encode year by first grouping movies into a new feature which will represent a range of years (e.g. 1975-1999 movies) and placing movies into that range if their year is within it, and one-hot encoding each entry of the list of ranges (e.g. 1950-1974, 1975-1999, 2000-2022).

We are going to eliminate all outliers from the year, runtime, rating, votes, and gross income categories. In order to do so we will discard all dataset outside of 1.5*IQR above and below the upper and lower quartiles. 

We will check for multicollinearity using a correlation matrix. We will eliminate values with correlation coefficients greater than 0.7.

# Data Cleaning
## Imports

In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.svm import SVR
# from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os

## Read In Data

In [2]:
path = r'./data'                     # use your path
files = glob.glob(os.path.join(path, "*.csv"))  

data_frames = []

for file in files:
    frame = pd.read_csv(file)
    frame['genre'] = frame['genre'].apply(lambda x: str(file))
    data_frames.append(frame)

    
df = pd.concat(data_frames, ignore_index=True)
df.head()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt9114286,Black Panther: Wakanda Forever,2022,PG-13,161 min,./data\action.csv,6.9,The people of Wakanda fight to protect their h...,Ryan Coogler,/name/nm3363032/,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri...","/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN
1,tt1630029,Avatar: The Way of Water,2022,PG-13,192 min,./data\action.csv,7.8,Jake Sully lives with his newfound family form...,James Cameron,/name/nm0000116/,"Sam Worthington, \nZoe Saldana, \nSigourney We...","/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN
2,tt5884796,Plane,2023,R,107 min,./data\action.csv,6.5,A pilot finds himself caught in a war zone aft...,Jean-François Richet,/name/nm0724938/,"Gerard Butler, \nMike Colter, \nTony Goldwyn, ...","/name/nm0124930/,/name/nm1591496/,/name/nm0001...",26220.0,NaN
3,tt6710474,Everything Everywhere All at Once,2022,R,139 min,./data\action.csv,8.0,A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert",/name/nm3453283/,"Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu...","/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN
4,tt5433140,Fast X,2023,NaN,NaN,./data\action.csv,NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN


## Remove NULL Values

In [10]:
df = df.dropna(subset=['year', 'certificate', 'runtime', 'rating']).reset_index(drop=True)
len(df)

5038

## Format Columns

In [4]:
#Format runtimes into floats
def convert_runtime(string):
    return float(string.split(' ')[0])
    
df['runtime'] = df['runtime'].apply(convert_runtime)
df = df[df['runtime'] < 300]


#Format genre by using only first value
def genre_parse(genre):
    genre = str(genre).split("\\")
    genre = genre[1].split('.')[0]
    
    genre = genre.lower()
    return genre[:3]
  
df['genre']=df['genre'].apply(genre_parse)
df = df.drop(df.loc[(df['genre'] == 'musical') | (df['genre'] == 'romance') | (df['genre'] == 'biography') | (df['genre']=='animation')].index)

# Format Certificate filter out all data points to only include movies with the G, PG, PG-13, R, and NC-17 ratings, switch Not Rated to NR
def convert_unrated(certificate):
    if certificate == 'Not Rated':
        return 'NR'
    else:
        return certificate
    
df=df.loc[(df['certificate'] == 'R') | (df['certificate'] =='PG-13') | (df['certificate'] == 'PG') | (df['certificate'] == 'Not Rated') | (df['certificate'] =='X') | (df['certificate'] == 'NC-17')]
df['certificate']=df['certificate'].apply(convert_unrated)

#Format years into ints and include only the past 25 years worth of data. 
df['year'] = df['year'].astype(int)
df = df.loc[(df['year'] >= 1997)].reset_index(drop=True)

df.head()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt1825683,Black Panther,2018,PG-13,134.0,act,7.3,"T'Challa, heir to the hidden but advanced king...",Ryan Coogler,/name/nm3363032/,"Chadwick Boseman, \nMichael B. Jordan, \nLupit...","/name/nm1569276/,/name/nm0430107/,/name/nm2143...",785813.0,700059566.0
1,tt0499549,Avatar,2009,PG-13,162.0,act,7.9,A paraplegic Marine dispatched to the moon Pan...,James Cameron,/name/nm0000116/,"Sam Worthington, \nZoe Saldana, \nSigourney We...","/name/nm0941777/,/name/nm0757855/,/name/nm0000...",1322694.0,760507625.0
2,tt1392170,The Hunger Games,2012,PG-13,142.0,act,7.2,Katniss Everdeen voluntarily takes her younger...,Gary Ross,/name/nm0002657/,"Jennifer Lawrence, \nJosh Hutcherson, \nLiam H...","/name/nm2225369/,/name/nm1242688/,/name/nm2955...",927499.0,408010692.0
3,tt1160419,Dune,2021,PG-13,155.0,act,8.0,A noble family becomes embroiled in a war for ...,Denis Villeneuve,/name/nm0898288/,"Timothée Chalamet, \nRebecca Ferguson, \nZenda...","/name/nm3154303/,/name/nm0272581/,/name/nm3918...",649342.0,108327830.0
4,tt0120737,The Lord of the Rings: The Fellowship of the Ring,2001,PG-13,178.0,act,8.8,A meek Hobbit from the Shire and eight compani...,Peter Jackson,/name/nm0001392/,"Elijah Wood, \nIan McKellen, \nOrlando Bloom, ...","/name/nm0000704/,/name/nm0005212/,/name/nm0089...",1889727.0,315544750.0


# Data Engineering

## Remove Repeats and One Hot Encode

In [5]:
df_pre_eng = df
genres = df['genre'].unique()
for genre in genres:
    df[genre] = 0
names = df['movie_name'].unique()

for name in names:
    repeats = df.loc[(df['movie_name'] == name)]
    genres = []
    
    for index in list(repeats.index):
        genres.append(repeats.loc[index,'genre'])
    
    for genre in genres:
        df.loc[list(repeats.index)[0], genre] = 1
    
    df = df.drop(list(repeats.index)[1:])
    
df=df.drop(['movie_name', 'genre'], axis=1)
df.head()

,movie_id,year,certificate,runtime,rating,description,director,director_id,star,star_id,...,cri,fam,fan,his,hor,mys,sci,spo,thr,war
0,tt1825683,2018,PG-13,134.0,7.3,"T'Challa, heir to the hidden but advanced king...",Ryan Coogler,/name/nm3363032/,"Chadwick Boseman, \nMichael B. Jordan, \nLupit...","/name/nm1569276/,/name/nm0430107/,/name/nm2143...",...,0,0,0,0,0,0,1,0,0,0
1,tt0499549,2009,PG-13,162.0,7.9,A paraplegic Marine dispatched to the moon Pan...,James Cameron,/name/nm0000116/,"Sam Worthington, \nZoe Saldana, \nSigourney We...","/name/nm0941777/,/name/nm0757855/,/name/nm0000...",...,0,0,1,0,0,0,1,0,0,0
2,tt1392170,2012,PG-13,142.0,7.2,Katniss Everdeen voluntarily takes her younger...,Gary Ross,/name/nm0002657/,"Jennifer Lawrence, \nJosh Hutcherson, \nLiam H...","/name/nm2225369/,/name/nm1242688/,/name/nm2955...",...,0,0,0,0,0,0,1,0,1,0
3,tt1160419,2021,PG-13,155.0,8.0,A noble family becomes embroiled in a war for ...,Denis Villeneuve,/name/nm0898288/,"Timothée Chalamet, \nRebecca Ferguson, \nZenda...","/name/nm3154303/,/name/nm0272581/,/name/nm3918...",...,0,0,0,0,0,0,1,0,0,0
4,tt0120737,2001,PG-13,178.0,8.8,A meek Hobbit from the Shire and eight compani...,Peter Jackson,/name/nm0001392/,"Elijah Wood, \nIan McKellen, \nOrlando Bloom, ...","/name/nm0000704/,/name/nm0005212/,/name/nm0089...",...,0,0,1,0,0,0,0,0,0,0


In [ ]:
# scale down gross earnings from dollars to millions of dollars
df['gross(in $)'] = df['gross(in $)']/1e6

# EDA

In [ ]:
from matplotlib.gridspec import GridSpec
plt.figure(figsize=(40,40))
plt.rc('font', size=50)          # controls default text sizes    # fontsize of the x and y labels
plt.rc('xtick', labelsize=50)    # fontsize of the tick labels
plt.rc('ytick', labelsize=50) 
fig, (ax1, ax2, ax3) = plt.subplots(3, 3, gridspec_kw={'wspace':1/8, 'hspace':1/8})

df['year'].hist(bins=range(1997, 2022, 4), ax=ax1[0])
ax1[0].set_xlabel('Year')
ax1[0].set_ylabel('Count')
ax1[0].set_title('Distribution of Year')

df['certificate'].hist(ax=ax1[1])
ax1[1].set_title('Distribution of Certificate')
ax1[1].set_xlabel('Certificate')
ax1[1].set_ylabel('Count')

df['runtime'].hist(bins=range(0, 300, 30), ax=ax1[2])
ax1[2].set_xlabel('Minutes')
ax1[2].set_ylabel('Count')
ax1[2].set_title('Distribution of Runtime')

df_pre_eng['genre'].hist(ax=ax2[0],figsize=(80,50))
ax2[0].set_title('Distribution of Genre')
ax2[0].set_xlabel('Genre')
ax2[0].set_ylabel('Count')

df['rating'].hist(ax=ax2[1])
ax2[1].set_title('Distribution of Rating')
ax2[1].set_xlabel('Rating')
ax2[1].set_ylabel('Count')

df['votes'].hist(ax=ax2[2])
ax2[2].set_title('Distribution of Number of Votes')
ax2[2].set_xlabel('Number of Votes')
ax2[2].set_ylabel('Count')

df['gross(in $)'].hist(ax=ax3[0])
ax3[0].set_title('Distribution of Gross Income (Millions of $)')
ax3[0].set_xlabel('Gross Income (Millions of $)')
ax3[0].set_ylabel('Count')

plt.show()

print('Earliest Year:', df['year'].min())
print('Latest Year:', df['year'].max())

print(df_pre_eng['genre'].unique())

print('Shortest Movie Runtime:', df['runtime'].min(), 'min')
print('Longest Movie Runtime:', df['runtime'].max(), 'min')
print('Average Movie Runtime:', df['runtime'].mean(), 'min')

In [ ]:
import seaborn as sns
plt.figure(figsize=(60,40))
plt.rc('font', size=40)          # controls default text sizes    # fontsize of the x and y labels
plt.rc('xtick', labelsize=40)    # fontsize of the tick labels
plt.rc('ytick', labelsize=40) 
fig, (ax1, ax2, ax3) = plt.subplots(3, 2, gridspec_kw={'wspace':1/16, 'hspace':1/16}, figsize=(80, 80))

sns.boxplot(df['year'],ax=ax1[0])
ax1[0].set_title('Years of Release')
ax1[0].set_ylabel('Year')

sns.boxplot(df['runtime'],ax=ax1[1])
ax1[1].set_title('Movie Runtimes')
ax1[1].set_ylabel('Runtime')

sns.boxplot(df['rating'],ax=ax2[0])
ax2[0].set_title('Movie Rating Out of 10')
ax2[0].set_ylabel('Rating')

sns.boxplot(df['votes'],ax=ax2[1])
ax2[1].set_title('Number of Votes for Movie')
ax2[1].set_ylabel('Number of Votes')

sns.boxplot(df['gross(in $)'],ax=ax3[0])
ax3[0].set_title('Gross Income (Millions of $)')
ax3[0].set_ylabel('Gross Income (Millions of $)')

In [ ]:
def outliers(ls):
    q3= ls.quantile(0.75)
    q1= ls.quantile(0.25)
    IQR=q3-q1
    outliers_upper = ls.loc[ls > q3 + 1.5 * IQR]
    outliers_lower = df['runtime'].loc[df['runtime'] < q1 - 1.5 * IQR]
    num_outliers_runtime = len(outliers_lower)+len(outliers_upper)
    return num_outliers_runtime

In [ ]:
print("Number of outliers detected in year data: " + str(outliers(df['year'])))
print("Number of outliers detected in runtime data: " + str(outliers(df['runtime'])))
print("Number of outliers detected in rating data: " + str(outliers(df['rating'])))
print("Number of outliers detected in votes data: " + str(outliers(df['votes'])))
print("Number of outliers detected in gross income data: " + str(outliers(df['gross(in $)'])))

In [ ]:
import seaborn as sns
# Correlation Matrix, dropping Film-Noir since none are in our dataframe anymore from the last 25 years
sns.set(rc={'figure.figsize':(12, 8)})
sns.heatmap(df.corr(), annot=True)

# Proposed Solution

We are going to use a few algorithms to approach the solution to this problem, but they are subsets of linear and nonlinear regression. This can be achieved through the use of sklearn's linear_model.LinearRegression (and its relatives Ridge and LASSO regression). In addition to this, we will utilize a decision tree model as well and utilize the machine learning model that scores better on our performance metrics. Our problem is clearly geared towards a regression solution as we aim to predict gross earnings from our features, instead of a classification problem in which logistic regression would be preferred. To test our solution, we will run cross-validation on our dataset to generate an estimate of our model's performance. A possible benchmark is simply guessing the mean gross box office earnings.


Because our problem has a very simple goal to achieve, we want to introduce nuance via complex approaches to analyzing the algorithms we use. We’ll use a few different standard model approaches including Gradient Boosting and Random Forest, as well as more complex approaches such as XGBoost and compare the results across each model. Again, because of the simplicity of our problem, we can also extend our analysis to using some hyperparameter optimization techniques, but since models like XGBoost get exceedingly complicated in the number of hyperparameters, we will likely choose instead to do random search and hope to establish good baselines for each approach, especially since we are most likely to use stratified k-fold cross-validation in order to preserve (for example) genre ratio while having a reasonably sized test set (given that our dataset has over 18000 examples). 

# Evaluation Metrics

To predict the box office gross income of movies based on their year, certificate, runtime, genre, rating and total number of votes, we will be potentially using the following metrics. 

Mean Absolute Error to measure the average absolute difference between the predicted box office gross and the actual box office gross across all the movies.

MAE = (1 / n) * sum(abs(y_true - y_pred))

Mean Squared Error and Root Mean Squared Error to measure the square root of the average squared difference between the predicted box office gross and the actual box office gross.

MSE = (1 / n) * sum((y_true - y_pred)^2)

RMSE = sqrt((1 / n) * sum((y_true - y_pred)^2))

And R2 Score to measure how well the regression model fits the actual data

R2 Score = 1 - (sum((y_true - y_pred)^2) / sum((y_true - mean(y_true))^2))

y_true = actual box office gross

y_pred = predicted box office gross

# Results

You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Maybe you include a learning curve to show whether you have enough data to do train/validate/test split or have to go to k-folds or LOOCV or ???

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5 

Maybe you do model selection again, but using a different kind of metric than before?



In [ ]:
features = list(df.columns)
X_train, X_test, y_train, y_test = train_test_split(df[[f for f in features if f not in ['gross(in $)']]]
                                                    , df['gross(in $)'], test_size=0.2, random_state=42)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax[0].hist(X_train['rating'], bins=20)
ax[0].set_title('Training Set')
ax[0].set_xlabel('Rating')
ax[0].set_ylabel('Frequency')
ax[1].hist(X_test['rating'], bins=20)
ax[1].set_title('Test Set')
ax[1].set_xlabel('Rating')
ax[1].set_ylabel('Frequency')
plt.show()

In [ ]:
num_scaler = StandardScaler()
ohe_encoder = OneHotEncoder()

In [ ]:
X_train_num = X_train[[f for f in features if f not in ['gross(in $)', 'certificate']]]
X_train_cat = X_train[['certificate']]

X_train_num_scaled = num_scaler.fit_transform(X_train_num)
X_train_cat_encoded = ohe_encoder.fit_transform(X_train_cat)

In [ ]:
X_train_preprocessed = pd.concat([
    pd.DataFrame(X_train_num_scaled, columns=[f for f in features if f not in ['gross(in $)', 'certificate']]),
    pd.DataFrame(X_train_cat_encoded.toarray(), columns=ohe_encoder.get_feature_names(['certificate']))
], axis=1)

In [ ]:
X_test_num = X_test[[f for f in features if f not in ['gross(in $)', 'certificate']]]
X_test_cat = X_test[['certificate']]

X_test_num_scaled = num_scaler.transform(X_test_num)
X_test_cat_encoded = ohe_encoder.transform(X_test_cat)

In [ ]:
X_test_preprocessed = pd.concat([
    pd.DataFrame(X_test_num_scaled, columns=[f for f in features if f not in ['gross(in $)', 'certificate']]),
    pd.DataFrame(X_test_cat_encoded.toarray(), columns=ohe_encoder.get_feature_names(['certificate']))
], axis=1)

In [ ]:
model = LinearRegression()
model.fit(X_train_preprocessed, y_train)
y_pred = model.predict(pd.concat([pd.DataFrame(X_test_num_scaled, columns=[f for f in features if f not in ['gross(in $)', 'certificate']]), 
                                  pd.DataFrame(X_test_cat_encoded.toarray(), columns=ohe_encoder.get_feature_names(['certificate']))], axis=1))

print('R2 score:', r2_score(y_test, y_pred))
print('Mean squared error:', mean_squared_error(y_test, y_pred))
print('Mean absolute error:', mean_absolute_error(y_test, y_pred))

<font size="5">Linear Regression</font>

In [ ]:
model = LinearRegression()
model.fit(X_train_preprocessed, y_train)
y_pred = model.predict(pd.concat([pd.DataFrame(X_test_num_scaled, columns=[f for f in features if f not in ['gross(in $)', 'certificate']]), 
                                  pd.DataFrame(X_test_cat_encoded.toarray(), columns=ohe_encoder.get_feature_names(['certificate']))], axis=1))

print('R2 score:', r2_score(y_test, y_pred))
print('Mean squared error:', mean_squared_error(y_test, y_pred))
print('Mean absolute error:', mean_absolute_error(y_test, y_pred))

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Gross Box Office Earnings")
plt.ylabel("Predicted Gross Box Office Earnings")
plt.show()

<font size="5">Decision Tree</font>

In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train_preprocessed, y_train)
y_pred = model.predict(X_test_preprocessed)

print('R2 score:', r2_score(y_test, y_pred))
print('Mean squared error:', mean_squared_error(y_test, y_pred))
print('Mean absolute error:', mean_absolute_error(y_test, y_pred))

<font size="8">Using Year as Categorical (5-year increment)</font>

In [ ]:
df['year_category'] = (((df['year'] - 1910) // 5) * 5 + 1910) // 10 * 10
df['year_category'] = df['year_category'].apply(lambda x: str(x) + '-' + str(x+4) if (x+3) < 2023 else str(x) + '-2022')
df

In [ ]:
year_counts = df['year_category'].value_counts().sort_index()
year_counts.plot.bar()
plt.title('Distribution of Year Category')
plt.xlabel('Year Category')
plt.ylabel('Count')
plt.show()

In [ ]:
features.append('year_category')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[[f for f in features if f not in ['year', 'gross(in $)']]]
                                                    , df['gross(in $)'], test_size=0.2, random_state=42)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax[0].hist(X_train['rating'], bins=20)
ax[0].set_title('Training Set')
ax[0].set_xlabel('Rating')
ax[0].set_ylabel('Frequency')
ax[1].hist(X_test['rating'], bins=20)
ax[1].set_title('Test Set')
ax[1].set_xlabel('Rating')
ax[1].set_ylabel('Frequency')
plt.show()

In [ ]:
# Distribution of Genres
# Extract the genre columns from the train and test sets
genre_cols = ['act', 'adv', 'cri', 'fam', 'fan', 'his', 'hor', 'mys', 'sci', 'spo', 'thr', 'war']
X_train_genre = X_train[genre_cols]
X_test_genre = X_test[genre_cols]

# Plot the histograms
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax[0].hist(X_train_genre.values, bins=20, label=genre_cols)
ax[0].set_title('Training Set')
ax[0].set_xlabel('Genre')
ax[0].set_ylabel('Frequency')
ax[0].legend()
ax[1].hist(X_test_genre.values, bins=20, label=genre_cols)
ax[1].set_title('Test Set')
ax[1].set_xlabel('Genre')
ax[1].set_ylabel('Frequency')
ax[1].legend()
plt.show()

In [ ]:
features

In [ ]:
X_train_num = X_train[[f for f in features if f not in ['year', 'certificate', 'gross(in $)', 'year_category']]]
X_train_cat = X_train[['certificate', 'year_category']]

X_train_num_scaled = num_scaler.fit_transform(X_train_num)
X_train_cat_encoded = ohe_encoder.fit_transform(X_train_cat)

In [ ]:
X_train_preprocessed = pd.concat([
    pd.DataFrame(X_train_num_scaled, columns=[f for f in features if f not in ['year', 'certificate', 'gross(in $)', 'year_category']]),
    pd.DataFrame(X_train_cat_encoded.toarray(), columns=ohe_encoder.get_feature_names(['certificate','year_category']))
], axis=1)

In [ ]:
X_test_num = X_test[[f for f in features if f not in ['year', 'certificate', 'gross(in $)', 'year_category']]]
X_test_cat = X_test[['certificate', 'year_category',]]

X_test_num_scaled = num_scaler.transform(X_test_num)
X_test_cat_encoded = ohe_encoder.transform(X_test_cat)

In [ ]:
X_test_preprocessed = pd.concat([
    pd.DataFrame(X_test_num_scaled, columns=[f for f in features if f not in ['year', 'certificate', 'gross(in $)', 'year_category']]),
    pd.DataFrame(X_test_cat_encoded.toarray(), columns=ohe_encoder.get_feature_names(['certificate', 'year_category']))
], axis=1)

## Linear Regression

In [ ]:
model = LinearRegression()
model.fit(X_train_preprocessed, y_train)
y_pred = model.predict(pd.concat([pd.DataFrame(X_test_num_scaled, columns=[f for f in features if f not in ['year', 'certificate', 'gross(in $)', 'year_category']]), 
                                  pd.DataFrame(X_test_cat_encoded.toarray(), columns=ohe_encoder.get_feature_names(['certificate', 'year_category']))], axis=1))

print('R2 score:', r2_score(y_test, y_pred))
print('Mean squared error:', mean_squared_error(y_test, y_pred))
print('Mean absolute error:', mean_absolute_error(y_test, y_pred))

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Gross Box Office Earnings")
plt.ylabel("Predicted Gross Box Office Earnings")
plt.show()

## Decision Tree

In [ ]:
model = DecisionTreeRegressor(max_depth=5)
model.fit(X_train_preprocessed, y_train)
y_pred= model.predict(pd.concat([pd.DataFrame(X_test_num_scaled, columns=[f for f in features if f not in ['year', 'certificate', 'gross(in $)', 'year_category']]),
                                  pd.DataFrame(X_test_cat_encoded.toarray(), columns=ohe_encoder.get_feature_names(['certificate', 'year_category']))], axis=1))

feature_names = X_train_preprocessed.columns
plt.figure(figsize=(50, 10))
plot_tree(model, filled=True, feature_names=feature_names)
plt.show()

print('R2 score:', r2_score(y_test, y_pred))
print('Mean squared error:', mean_squared_error(y_test, y_pred))
print('Mean absolute error:', mean_absolute_error(y_test, y_pred))

In [ ]:
model = SVR(kernel='rbf', C=100, gamma='auto')
model.fit(X_train_preprocessed, y_train)
y_pred = model.predict(pd.concat([pd.DataFrame(X_test_num_scaled, columns=[f for f in features if f not in ['year', 'certificate', 'gross(in $)', 'year_category']]), 
                                  pd.DataFrame(X_test_cat_encoded.toarray(), columns=ohe_encoder.get_feature_names(['certificate', 'year_category']))], axis=1))

print('R2 score:', r2_score(y_test, y_pred))
print('Mean squared error:', mean_squared_error(y_test, y_pred))
print('Mean absolute error:', mean_absolute_error(y_test, y_pred))

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train_preprocessed, y_train)

y_pred = rf.predict(pd.concat([pd.DataFrame(X_test_num_scaled, columns=[f for f in features if f not in ['year', 'certificate', 'gross(in $)', 'year_category']]), 
                                  pd.DataFrame(X_test_cat_encoded.toarray(), columns=ohe_encoder.get_feature_names(['certificate', 'year_category']))], axis=1))
mse = mean_squared_error(y_test, y_pred)
print(mse)

## XGB Boost

In [ ]:
model = xgb.XGBRegressor()
model.fit(X_train_preprocessed, y_train)

y_pred = model.predict(X_test_preprocessed)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: ", rmse)

# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

In [ ]:
xgb.plot_importance(model)
plt.show()

# Hyperparameter Optimization on Gradient Boost

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

gbr = GradientBoostingRegressor()
grid = dict()
grid['n_estimators'] = [10, 50, 100, 1000]
grid['learning_rate'] = [0.001, 0.005, 0.01, 0.05, 0.1]
grid['subsample'] = [0.5, 0.7, 1.0]
grid['max_depth'] = [3, 5, 7]

grid_search = GridSearchCV(estimator=gbr, param_grid=grid, cv=10)
# search over our floating values
grid_result = grid_search.fit(X=df[[f for f in features if f not in ['year', 'certificate', 'gross(in $)', 'year_category']]], y=df['gross(in $)'])

In [ ]:
print('OPTIMAL: %f with params: %s' % (grid_result.best_score_, grid_result.best_params_))

test_means = grid_result.cv_results_['mean_test_score']
test_stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for test_mean, test_std, param in zip(test_means, test_stds, params):
    print("%f (%f) with params: %r" % (test_mean, test_std, param))

OPTIMAL: 0.460124 with params: {'learning_rate': 0.005, 'max_depth': 5, 'n_estimators': 1000, 'subsample': 0.5}
-1.908803 (3.119256) with params: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 10, 'subsample': 0.5}
-1.908782 (3.118924) with params: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 10, 'subsample': 0.7}
-1.908568 (3.118712) with params: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 10, 'subsample': 1.0}
-1.748135 (2.908756) with params: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.5}
-1.747606 (2.907288) with params: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.7}
-1.748217 (2.907749) with params: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1.0}
-1.562902 (2.666039) with params: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}
-1.563336 (2.666657) with params: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100, 

# Discussion

### Interpreting the result

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.

### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   

### Ethics & Privacy

The IMDB dataset is downloaded from the Kaggle website with no personal nor sensitive information included from human participants. All the variables in the dataset are available to public on IMDB website in which we can eliminate the concern for data privacy and misuse of the data. The movie ratings and box office gross are largely influenced by the viewers and fans of particular franchises and movie stars which may be biased, however, since we are using a large dataset with a huge number of ratings, the bias could be neglected or will be evaluated and addressed if the concerns were to arise. Our final product will be available to students of current COGS118A class, future prospective students and potentially to public and we will carefully monitor and identify the unintended use of our project.

### Conclusion

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

# Footnotes
<a name="doshinote"></a>1.[^](#doshi): Doshi, Lyric, et al. Predicting Movie Prices through Dynamic Social Network Analysis - Core. https://core.ac.uk/download/pdf/82778929.pdf. <br> 

<a name="dhirnote"></a>2.[^](#dhir): Dhir R, Raj A (2018) Movie success prediction using machine learning algorithms and their comparison. In: 2018 First International Conference on Secure Cyber Computing and Communication (ICSCCC), IEEE, pp 385–390. <br>

<a name='abidinote'></a>3.[^](#abidi): Abidi, S.M.R., Xu, Y., Ni, J. et al. Popularity prediction of movies: from statistical modeling to machine learning techniques. Multimed Tools Appl 79, 35583–35617 (2020). https://doi.org/10.1007/s11042-019-08546-5. <br>